# Training of complex Neural Network

In [ ]:
%load_ext autoreload
%autoreload 2

# external imports
import numpy as np
import pandas as pd
import seaborn as sb
import gensim
from gensim.scripts.glove2word2vec import glove2word2vec
import csv
import scipy
import os.path
import sklearn as sk
import keras
import random
import pickle
import gc
import time

# internal imports
import helpers as HL
import glove_module as GV
import neural_nets as NN
import tokenizing as TO



# Constants
DATA_FOLDER = "gensim_data_folder"
DATA_25DIM = DATA_FOLDER + "/gensim_glove_vectors_25dim.txt"
DATA_50DIM = DATA_FOLDER + "/gensim_glove_vectors_50dim.txt"
DATA_100DIM = DATA_FOLDER + "/gensim_glove_vectors_100dim.txt"
DATA_200DIM = DATA_FOLDER + "/gensim_glove_vectors_200dim.txt"

#### Import corpus

In [ ]:
# Loading corpus

awesome_corpus = pickle.load( open( "stopword100_corpus_n2_SHM_E_SN_H_HK.pkl", "rb" ) )
print(len(awesome_corpus))

nr_pos_tweets = 1250000
nr_neg_tweets = 1250000
total_training_tweets = 2500000

#### Import global vectors

In [ ]:
# uses the created gensim-.txt file to create the word2vec so one can operate on it
global_vectors = GV.make_glove(DATA_200DIM)

#### Vectorize corpus

In [ ]:
start = time.time()

###### build vectors of all the tweets ######
num_of_dim = global_vectors.syn0.shape[1]
# seperate traindata and testdata
train_corpus = processed_corpus[:total_training_tweets:] 
predict_corpus = processed_corpus[total_training_tweets::]
del processed_corpus

# Build a vector of all the words in a tweet
vectors = np.zeros(len(train_corpus), dtype=object)
for i, doc in enumerate(train_corpus):
    if (i % 50000) == 0:
        print("tweets processed: %.0f  of total number of tweets: %.0f" % (i,len(train_corpus)))
    vectors[i] = GV.buildWordVector(doc, num_of_dim, global_vectors)
del global_vectors
del doc
print("done with making the trainvectors")

train_document_vecz = np.concatenate(vectors)
del vectors
print("done with concatenating the trainvectors")

train_document_vecs = sk.preprocessing.scale(train_document_vecz)
del train_document_vecz
print("done with scaling the trainvectors")

labels = GV.create_labels(total_training_tweets, nr_pos_tweets)
print("done with creating the labels")
print("time used one the ordeal:", time.time() - start)

#############################################

#### Define the neural network model

In [ ]:
# Defing model :)
input_dimensions = train_document_vecs.shape[1]
width = 500
depth = 2
epochs = 60
n_folds = 2
split = 0.9
dropout_rate=0.4
funnel=0.3

#model = NN.deep_HB(input_dimensions)
model = NN.dynamic_dense(input_dimensions, width, depth, dropout_rate=dropout_rate, activation='relu', funnel=funnel)
print(model.summary())

#### Train model on dataset
This can take a while, it should stop with early stopping(patience=10!), but it can can be stopped prematurely by Interrupting kernel. Then it return the last model it was working on(BUT NOT NECESSARILY THE BEST). Use the "train_NN_dynamic_model.hdf5" that is saved by ModelCheckpoint. Maybe try out once before running it for a long time

In [ ]:
improved_model, history = GV.train_NN(model, train_document_vecs, labels, epochs=1)
# backuppickle in case something goes wrong
improved_model.save('Backup_of_further_training_model.h5')

## Submit Kaggle when you are happy with your model

In [ ]:
# FOR THE KAGGLE SUBMISSION
test_document_vecs = np.concatenate([GV.buildWordVector(doc, num_of_dim, global_vectors) for doc in predict_corpus])
test_document_vecs = sk.preprocessing.scale(test_document_vecs)

print("Hello world")
pred=model.predict(test_document_vecs)

pred_ones=[]
for i in pred:
    if i> 0.5:
        pred_ones.append(1)
    else:
        pred_ones.append(-1)

#CREATING SUBMISSION
ids = list(range(1,10000+1))
HL.create_csv_submission(ids, pred_ones,"best_proc_corpus_dynamic_dense.txt")